In [ ]:
#!pip install tensorflow-gpu==1.15.0 tensorflow==1.15.0 stable-baselines gym-anytrading-debug gym
#!pip install finta

In [ ]:
# Gym stuff
import gym
import gym_anytrading_debug

# Stable baselines - rl stuff
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import os
import logging
from gym_anytrading_debug.envs import MyStocksEnv
from finta import TA

# 1. Load data to train with
 Bring in Crypto ETH/USDT Data 
https://www.cryptodatadownload.com/data/binance/

In [ ]:
from csv_import_helper import *
#df = import_csv_from_cryptodatadownload('data/Binance_ETHUSDT_d.csv')
df = import_example_sin_data(2000)

# 2. Build Environment

In [ ]:
nb_data_rows = len(df.index)
print(nb_data_rows)
window_size = 12

In [ ]:
df[df.index.duplicated()]

## 2.1.3. Calculate custom indicators SMA, RSI and OBV

In [ ]:
df['SMA'] = TA.SMA(df, 12)
df['RSI'] = TA.RSI(df)
df['OBV'] = TA.OBV(df)
df.fillna(0, inplace=True)

In [ ]:
df_temp = df[["SMA", "RSI"]]
df_temp.plot()
#df.plot()

## 2.1.4. Create New Environments

In [ ]:
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['low','volume','SMA', 'RSI', 'OBV']].to_numpy()[start:end]
    #signal_features = env.df.loc[:, ['low','volume','tradecount','MACD', 'SIGNAL']].to_numpy()[start:end]
    return prices, signal_features

In [ ]:
class MyCustomEnv(MyStocksEnv):
    init_data = add_signals
    
env = MyCustomEnv(df=df, window_size=window_size, frame_bound=(501, nb_data_rows), debug=False)

In [ ]:
env_maker = lambda: env
env = DummyVecEnv([env_maker])

In [ ]:
import os
create_new_model = False
if create_new_model:
    model = A2C('MlpLstmPolicy', env, verbose=1) 
else:
    model_path = os.path.join('SavedModels', 'A2C_TM1_ETH_D_w12_E47')
    model = A2C.load(model_path, env)

In [ ]:
#model.learn(total_timesteps=100000)

# 4. Evaluation

In [ ]:
import logging

#env = MyCustomEnv(df=df, window_size=window_size, frame_bound=(501, nb_data_rows), debug=True)
envTest = MyCustomEnv(df=df, window_size=window_size, frame_bound=(400, nb_data_rows), debug=True, log_level=logging.DEBUG)
obs = envTest.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, rewards, done, info = envTest.step(action)
    if done:
        print("info", info)
        print("Performance: {0}".format(info['performance']))
        break

In [ ]:
#plt.cla()
envTest.render_all()
#plt.figure(figsize=(25,6))
#plt.show()

In [ ]:
import logging

learn = False
eval_nb = 10

file_name = 'A2C_Trading_Model1_Daily_winsize12_V1'

model_path = os.path.join('SavedModels', file_name)

episode = 0
training_file_name = 'A2C_TM1_ETH_D_w12_'

#Training loop
while learn:

    model = A2C.load(model_path, env)

    # train
    model.learn(total_timesteps=100000)

    # evaluate for eval_nb times, perfomance must be always above 1
    perfomance_results = []

    envTest = MyCustomEnv(df=df, window_size=window_size, frame_bound=(13, nb_data_rows), debug=True, log_level=logging.DEBUG)
    for i in range(0, eval_nb):
        obs = envTest.reset()
        performance = 1.0
        print("Start eval nb. {0}".format(i))
        while True: 
            obs = obs[np.newaxis, ...]
            action, _states = model.predict(obs)
            obs, rewards, done, info = envTest.step(action)
            if done:
                print("info", info)
                perfomance_results.append(info['performance'])
                break

    # check results
    print(perfomance_results)
    if all(x > 3.0 for x in perfomance_results):
        print("Yes, mission acomplished!!")
        learn = False
    else:
        print("No, train further")
        
    #save model
    temp_file_name = "{0}E{1}".format(training_file_name, episode)
    model_path = os.path.join('SavedModels', temp_file_name)
    model.save(model_path)
    
    episode += 1

In [ ]:
import os
model_path = os.path.join('SavedModels', 'A2C_Trading_Model1_Daily_winsize12_tempSave')
#model.save(model_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
in_array = np.linspace(0, 150, 2000)
out_array = (np.sin(in_array)*40) + 100

np.savetxt("foo.csv", out_array, delimiter=",")
 
print("in_array : ", in_array)
print("\nout_array : ", out_array)
 
# red for numpy.sin()
plt.figure(figsize=(25,6))
plt.plot(in_array, out_array, color = 'red', marker = "o")
plt.title("numpy.sin()")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()